This notebook is an exploration of how to access CPES (cadastral data) for Western Australia.

Each state manages their official spatial cadastral database separately, so there is no Australia-wide database we can access unfortunetly.

The data in this notebook are the publicly available ones from DPIRD - they are derived from the Cadastral Database, but are not directly sourced from it. So, the data may be different from the official records.

- https://catalogue.data.wa.gov.au/dataset/cadastre-address-lgate-002
- https://catalogue.data.wa.gov.au/dataset/cadastre-no-attributes-lgate-001
- https://catalogue.data.wa.gov.au/dataset/cadastre-polygon/resource/31a6a206-2b82-425f-84bf-bd4537a260f0
- https://catalogue.data.wa.gov.au/dataset/client-property-event-system-properties
- mixed access: https://catalogue.data.wa.gov.au/dataset/historical-cadastre-2023-polygon-lgate

### Consideratins
- Try filtering by bbox
- May also be able to fitler and return data based on attributes - e.g. locality, road name etc.

In [1]:
%pip install --upgrade hvplot holoviews panel jupyter_bokeh bokeh -q
%pip install dask[dataframe] -q
%pip install geojson -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import io
from io import BytesIO
import json
import requests
from urllib import parse, request
import holoviews as hv
import hvplot.pandas
import bokeh
import panel as pn
import geopandas as gpd
from shapely.geometry import box
from owslib.fes import *
from owslib.etree import etree
from owslib.wfs import WebFeatureService
from owslib.wms import WebMapService

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, interact

from fiona import BytesCollection 
from shapely.geometry import box 
from lxml import etree 

hv.extension("bokeh")
pn.extension()

In [3]:
print('Panel version:', pn.__version__)
print('Bokeh version:', bokeh.__version__)

Panel version: 1.4.4
Bokeh version: 3.4.1


In [4]:
# import area of interest

input_data_dir = "/workspace/notebooks/sandbox/data/input-data/paddock-boundaries"
output_data_dir = "/workspace/notebooks/sandbox/data/output-data/terrawise"

input_aoi = os.path.join(input_data_dir, "paddock-boundaries.shp")

aoi_gpd = gpd.read_file(input_aoi)

In [5]:
target_crs = 'EPSG:4326'  # WGS 84
DPIRD_crs = 'EPSG:7844' #this is GDA2020

if aoi_gpd.crs != DPIRD_crs:
    aoi_gpd = aoi_gpd.to_crs(DPIRD_crs)
    


bbox = list(aoi_gpd.total_bounds)

#NOTE: WFS handle coordinates differently to standard bounding boxes (yes, it's annoying as hell). 
#Convert a standard bounding box to the correct format before trying to use it in a WFS request.
lat_long_bounds = (bbox[1], bbox[0], bbox[3], bbox[2])

print(f"The original bounding box: {bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}")
print(f"The lat/long bounding box: {lat_long_bounds}")

The original bounding box: 114.87989378387825,-28.0458056740587,114.94593929947102,-27.98170548044053
The lat/long bounding box: (-28.0458056740587, 114.87989378387825, -27.98170548044053, 114.94593929947102)


In [6]:
#set up access to DPIRD WFS endpoint
wfs_resource_url_cadastre_address = "https://public-services.slip.wa.gov.au/public/services/SLIP_Public_Services/Places_and_Addresses_WFS/MapServer/WFSServer"
wfs_resource_url_cadastre_simple = "https://public-services.slip.wa.gov.au/public/services/SLIP_Public_Services/Property_and_Planning_WFS/MapServer/WFSServer"
wfs_resource_url_cadastre_2023_public = "https://public-services.slip.wa.gov.au/public/services/SLIP_Public_Services/Historical_Cadastre_WFS/MapServer/WFSServer"

wfs_cadastre_address = WebFeatureService(url=wfs_resource_url_cadastre_address, version='2.0.0')
wfs_cadastre_simple = WebFeatureService(url=wfs_resource_url_cadastre_simple, version='2.0.0')
wfs_cadastre_2023_public = WebFeatureService(url=wfs_resource_url_cadastre_2023_public, version='2.0.0')


In [7]:
# TODO: use panel to enable user to select which WFS layer to display

print(f" WFS Cadastral Address title: {wfs_cadastre_address.identification.title}")
print(f" WFS Cadastral Address operations available: {[operation.name for operation in wfs_cadastre_address.operations]}")
print(f"Get WFS 'get feature' methods: {wfs_cadastre_address.getOperationByName('GetFeature').methods}")
print(f"Get WFS 'get feature' format options: {wfs_cadastre_address.getOperationByName('GetFeature').formatOptions}")


 WFS Cadastral Address title: WFS
 WFS Cadastral Address operations available: ['GetCapabilities', 'DescribeFeatureType', 'GetPropertyValue', 'GetFeature', 'GetGmlObject', 'ListStoredQueries', 'DescribeStoredQueries', 'ImplementsBasicWFS', 'ImplementsTransactionalWFS', 'ImplementsLockingWFS', 'KVPEncoding', 'XMLEncoding', 'SOAPEncoding', 'ImplementsInheritance', 'ImplementsRemoteResolve', 'ImplementsResultPaging', 'ImplementsStandardJoins', 'ImplementsSpatialJoins', 'ImplementsTemporalJoins', 'ImplementsFeatureVersioning', 'ManageStoredQueries', 'CountDefault']
Get WFS 'get feature' methods: [{'constraints': [], 'type': 'Get', 'url': 'https://public-gr-admin.slip.wa.gov.au:443/public/services/SLIP_Public_Services/Places_and_Addresses_WFS/MapServer/WFSServer?'}, {'constraints': [], 'type': 'Post', 'url': 'https://public-gr-admin.slip.wa.gov.au:443/public/services/SLIP_Public_Services/Places_and_Addresses_WFS/MapServer/WFSServer'}]
Get WFS 'get feature' format options: ['text/xml']


In [8]:
list(wfs_cadastre_address.contents)

feature_type = list(wfs_cadastre_address.contents.keys())

for feature_type in wfs_cadastre_address.contents:
    print(feature_type)

esri:Geographic_Names__GEONOMA___LGATE-013_
esri:Beach_Emergency_Numbers__BEN__Signage__DPIRD-054_
esri:Cadastre_Address__LGATE-002_


In [9]:
list(wfs_cadastre_simple.contents)

feature_type = list(wfs_cadastre_simple.contents.keys())

for feature_type in wfs_cadastre_simple.contents:
    print(feature_type)

esri:Clearing_Regulations_-_Schedule_One_Areas__DWER-057_
esri:Swan_and_Canning_River_-_Development_Control_Area__DBCA-028_
esri:DBCA_-_Legislated_Lands_and_Waters__DBCA-011_
esri:DBCA_-_Lands_of_Interest__DBCA-012_
esri:DBCA_Planning_Referrals_Contacts__DBCA-033_
esri:EPB_Local_Assesment_Port_Hedland_Port_2011__DWER-061_
esri:SWAN_Bioplan_Peel_Sector_2010__DWER-069_
esri:SWAN_Bioplan_Regionally_Significant_Natural_Areas_2010__DWER-070_
esri:State_Environment_Policy_Cockburn_Protection_Levels_2010__DWER-068_
esri:EPP_Goldfields_Residential_Areas_Sulphur_Dioxide_2003__DWER-063_
esri:Geodetic_Survey_Mark_Reference_Marks__Point___LGATE-199_
esri:Geodetic_Survey_Marks__Point___LGATE-076_
esri:Geodetic_Survey_Network_Observations__LGATE-261_
esri:Cadastre__No_Attributes___LGATE-001_
esri:State_Planning_Policy_2.4__Perth_Peel_Region__DMIRS-071_
esri:State_Planning_Policy_2.4__Extraction_Sites__DMIRS-072_
esri:State_Planning_Policy_2.4__Exclusion_Areas__DMIRS-073_
esri:State_Planning_Policy_2

In [10]:
list(wfs_cadastre_2023_public.contents)

feature_type = list(wfs_cadastre_2023_public.contents.keys())

print(len(feature_type))

for feature_type in wfs_cadastre_2023_public.contents:
    print(feature_type)

16
esri:Historical_Cadastre_2008__Polygon___LGATE-298_
esri:Historical_Cadastre_2009__Polygon___LGATE-297_
esri:Historical_Cadastre_2010__Polygon___LGATE-296_
esri:Historical_Cadastre_2011__Polygon___LGATE-295_
esri:Historical_Cadastre_2012__Polygon___LGATE-294_
esri:Historical_Cadastre_2013__Polygon___LGATE-293_
esri:Historical_Cadastre_2014__Polygon___LGATE-292_
esri:Historical_Cadastre_2015__Polygon___LGATE-291_
esri:Historical_Cadastre_2016__Polygon___LGATE-290_
esri:Historical_Cadastre_2017__Polygon___LGATE-289_
esri:Historical_Cadastre_2018__Polygon___LGATE-288_
esri:Historical_Cadastre_2019__Polygon___LGATE-299_
esri:Historical_Cadastre_2020__Polygon___LGATE-312_
esri:Historical_Cadastre_2021__Polygon___LGATE-337_
esri:Historical_Cadastre_2022__Polygon___LGATE-475_
esri:Historical_Cadastre_2023__Polygon___LGATE-476_


In [11]:
for feature_type in wfs_cadastre_address.contents:
    print(feature_type)

esri:Geographic_Names__GEONOMA___LGATE-013_
esri:Beach_Emergency_Numbers__BEN__Signage__DPIRD-054_
esri:Cadastre_Address__LGATE-002_


In [12]:
for feature_type in wfs_cadastre_simple.contents:
    print(feature_type)

esri:Clearing_Regulations_-_Schedule_One_Areas__DWER-057_
esri:Swan_and_Canning_River_-_Development_Control_Area__DBCA-028_
esri:DBCA_-_Legislated_Lands_and_Waters__DBCA-011_
esri:DBCA_-_Lands_of_Interest__DBCA-012_
esri:DBCA_Planning_Referrals_Contacts__DBCA-033_
esri:EPB_Local_Assesment_Port_Hedland_Port_2011__DWER-061_
esri:SWAN_Bioplan_Peel_Sector_2010__DWER-069_
esri:SWAN_Bioplan_Regionally_Significant_Natural_Areas_2010__DWER-070_
esri:State_Environment_Policy_Cockburn_Protection_Levels_2010__DWER-068_
esri:EPP_Goldfields_Residential_Areas_Sulphur_Dioxide_2003__DWER-063_
esri:Geodetic_Survey_Mark_Reference_Marks__Point___LGATE-199_
esri:Geodetic_Survey_Marks__Point___LGATE-076_
esri:Geodetic_Survey_Network_Observations__LGATE-261_
esri:Cadastre__No_Attributes___LGATE-001_
esri:State_Planning_Policy_2.4__Perth_Peel_Region__DMIRS-071_
esri:State_Planning_Policy_2.4__Extraction_Sites__DMIRS-072_
esri:State_Planning_Policy_2.4__Exclusion_Areas__DMIRS-073_
esri:State_Planning_Policy_2

In [13]:
for feature_type in wfs_cadastre_2023_public.contents:
    print(feature_type)

esri:Historical_Cadastre_2008__Polygon___LGATE-298_
esri:Historical_Cadastre_2009__Polygon___LGATE-297_
esri:Historical_Cadastre_2010__Polygon___LGATE-296_
esri:Historical_Cadastre_2011__Polygon___LGATE-295_
esri:Historical_Cadastre_2012__Polygon___LGATE-294_
esri:Historical_Cadastre_2013__Polygon___LGATE-293_
esri:Historical_Cadastre_2014__Polygon___LGATE-292_
esri:Historical_Cadastre_2015__Polygon___LGATE-291_
esri:Historical_Cadastre_2016__Polygon___LGATE-290_
esri:Historical_Cadastre_2017__Polygon___LGATE-289_
esri:Historical_Cadastre_2018__Polygon___LGATE-288_
esri:Historical_Cadastre_2019__Polygon___LGATE-299_
esri:Historical_Cadastre_2020__Polygon___LGATE-312_
esri:Historical_Cadastre_2021__Polygon___LGATE-337_
esri:Historical_Cadastre_2022__Polygon___LGATE-475_
esri:Historical_Cadastre_2023__Polygon___LGATE-476_


In [14]:
response = wfs_cadastre_address.getfeature(typename='esri:Cadastre_Address__LGATE-002_',
                          bbox=lat_long_bounds)

In [15]:
#response2 = wfs_cadastre_simple.getfeature(typename='esri:Geographic_Names__GEONOMA___LGATE-013_',
#                          bbox=lat_long_bounds)

In [16]:
response3 = wfs_cadastre_2023_public.getfeature(typename='esri:Historical_Cadastre_2023__Polygon___LGATE-476_',
                          bbox=lat_long_bounds)

In [17]:
# Why are we writing our data as an XML? Because for some reason, that's the only output format DPIRD SLIP will let us!
out = open('/workspace/notebooks/sandbox/data/output-data/terrawise/Cadastre_Address__LGATE-002_.xml', 'wb')
out.write(response.read())
out.close()

In [18]:
out3 = open('/workspace/notebooks/sandbox/data/output-data/terrawise/Historical_Cadastre_2023__Polygon___LGATE-476_.xml', 'wb')
out3.write(response3.read())
out3.close()

In [19]:
data = gpd.read_file('/workspace/notebooks/sandbox/data/output-data/terrawise/Cadastre_Address__LGATE-002_.xml')
data3 = gpd.read_file('/workspace/notebooks/sandbox/data/output-data/terrawise/Historical_Cadastre_2023__Polygon___LGATE-476_.xml')

In [20]:
data['full_address'] = data['road_number_1'].map(str) + ' ' + data['road_name'].map(str) + ' ' + data['road_type'].map(str)

data.head()

,gml_id,land_id,view_scale,OBJECTID,st_perimeter_shape_,road_number_type,road_number_1,road_name,road_type,road_suffix,locality,geometry,full_address
0,Cadastre_Address__LGATE-002_.60228,1237407,256K,60228,0.106501,None,NaN,None,None,None,None,"MULTIPOLYGON (((114.91377 -28.04123, 114.91526...",nan None None
1,Cadastre_Address__LGATE-002_.559133,1795341,256K,559133,0.109369,None,NaN,None,None,None,None,"MULTIPOLYGON (((114.90568 -28.04230, 114.90163...",nan None None
2,Cadastre_Address__LGATE-002_.756900,2023208,256K,756900,0.080604,None,NaN,None,None,None,None,"MULTIPOLYGON (((114.94192 -28.04124, 114.94192...",nan None None
3,Cadastre_Address__LGATE-002_.756901,2023209,256K,756901,0.077925,H,2720.0,BINNU,RD,E,BINNU,"MULTIPOLYGON (((114.94192 -28.04124, 114.94481...",2720.0 BINNU RD
4,Cadastre_Address__LGATE-002_.756905,2023214,256K,756905,0.080418,None,NaN,None,None,None,None,"MULTIPOLYGON (((114.92404 -28.03669, 114.92541...",nan None None


In [21]:
data3.head()

,gml_id,polygon_number,usage_code,usage_description,OBJECTID,st_area_shape_,st_perimeter_shape_,geometry
0,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005125,2,Land Act (Type 2),265342,0.001008,0.134148,"MULTIPOLYGON (((114.86789 -27.98143, 114.87255..."
1,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005129,2,Land Act (Type 2),265345,0.000466,0.102562,"MULTIPOLYGON (((114.90556 -27.98147, 114.91061..."
2,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005130,2,Land Act (Type 2),265346,0.000371,0.091087,"MULTIPOLYGON (((114.90556 -27.99375, 114.91009..."
3,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005131,6,Unallocated Crown Land (Type 3 V),265347,0.000002,0.005901,"MULTIPOLYGON (((114.90550 -28.04141, 114.90752..."
4,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005132,6,Unallocated Crown Land (Type 3 V),265348,0.000004,0.011699,"MULTIPOLYGON (((114.92005 -28.04105, 114.92186..."


In [22]:
data3 = data3.loc[(data3['usage_code'] == 1) | (data3['usage_code'] == 2)]
data3.head()


,gml_id,polygon_number,usage_code,usage_description,OBJECTID,st_area_shape_,st_perimeter_shape_,geometry
0,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005125,2,Land Act (Type 2),265342,0.001008,0.134148,"MULTIPOLYGON (((114.86789 -27.98143, 114.87255..."
1,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005129,2,Land Act (Type 2),265345,0.000466,0.102562,"MULTIPOLYGON (((114.90556 -27.98147, 114.91061..."
2,Historical_Cadastre_2023__Polygon___LGATE-476_...,1005130,2,Land Act (Type 2),265346,0.000371,0.091087,"MULTIPOLYGON (((114.90556 -27.99375, 114.91009..."
6,Historical_Cadastre_2023__Polygon___LGATE-476_...,1247708,1,Transfer of Land Act (Type 1),290982,0.000676,0.109369,"MULTIPOLYGON (((114.90568 -28.04230, 114.90163..."
7,Historical_Cadastre_2023__Polygon___LGATE-476_...,721786,1,Transfer of Land Act (Type 1),307493,0.000660,0.106494,"MULTIPOLYGON (((114.88948 -28.04500, 114.89433..."


In [23]:
height = 800
width = 800

aoi = aoi_gpd.hvplot(geo=True, tiles='EsriImagery', alpha=0, line_alpha=1, line_width=3, line_color='red', width=width)
cpes_address = data.hvplot(geo=True, tiles='EsriImagery', c="OBJECTID", alpha=0.8,  width=width, hover_cols=['gml_id', 'full_address'])
cpes_2023 = data3.hvplot(geo=True, tiles='EsriImagery', c="usage_description", alpha=0.8, width=width, hover_cols=['gml_id', 'polygon_number', 'usage_code', 'OBJECTID'])

cpes_address_map = cpes_address * aoi
cpes_2023_map = cpes_2023 * aoi

dashbaord = pn.Column(cpes_2023_map)
pn.panel(dashbaord).show()

Launching server at http://localhost:41413
